In [ ]:
## TODO:: check row fixen
## kunnen we af van de  verschillende online dingen?
## kunnen we af van jquery

In [9]:
import json
import numbers
import os
from string import Template
import tempfile

from IPython.display import IFrame

content = Template('''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
    <meta http-equiv="x-ua-compatible" content="ie=edge">
    <title>Title</title>
    <link rel="stylesheet" type="text/css" href="./css/styling.css">
    <link rel="stylesheet" type="text/css" href="./css/d3.parcoords.css">
    <link rel="stylesheet" type="text/css" href="./css/bootstrap.css">
    <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.1.1/jquery.min.js" integrity="sha384-3ceskX3iaEnIogmQchP8opvBy3Mi7Ce34nWjpBIwVTHfGYWQS9jwHDVRnpKKHJg7" crossorigin="anonymous"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/tether/1.3.7/js/tether.min.js" integrity="sha384-XTs3FgkjiBgo8qjEjBk0tGmf3wPrWtA6coPfQDfFEY8AnYJwjalXCiosYRBIBZX8" crossorigin="anonymous"></script>
    <script src="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0-alpha.5/js/bootstrap.min.js" integrity="sha384-BLiI7JTZm+JWlgKa0M0kGRpJbF2J8q+qreVrKBC47e3K6BW78kGLrCkeRX6I9RoK" crossorigin="anonymous"></script>
    <script src="./js/d3.js"></script>
    <script src="./js/d3.svg.multibrush.js"></script>
    <script src="./js/sylvester.js"></script>
    <script src="./js/underscore.js"></script>
    <script src="./js/slickgrid/jquery-1.7.min.js"></script>
    <script src="./js/underscore.js"></script>
    <script src="./js/underscore.math.js"></script>
    <script src="./js/d3.parcoords.js"></script>
    <script src="./js/d3-beeswarm.js"></script>
    <script src="http://d3js.org/queue.v1.min.js"></script>
    <script src="./js/canvas2svg.js"></script>
    <script src="./js/FileSaver.js"></script>
    <script src="./js/vkbeautify.0.99.00.beta.js"></script>
    <script src="./js/science.v1.js"></script>
</head>
<body>
    <div class="container">
        <div id="row1" class="row">
            <div class="col-xl-2">
            </div>
            <div id="input" class="col-xl-10">
            </div>
        </div>
        <div id="row2" class="row">
            <div class="col-lg-2 col-xl-2">
            </div>
            <div class="col-lg-10 col-xl-10">
                <div id="example" class="parcoords"></div>
            </div>
        </div>
        <div id="row3" class="row">
            <!--should perhaps be generated by javascript, so we can properly manage checked state-->
            <!--alternatively only create the event handlers in javascript, and then also ensure unchecked-->
            <!--http://www.w3schools.com/jquery/jquery_hide_show.asp we want to only show row once we select-->
            <!--in the outcome space, we also want to hide again if selection is disabled-->
            <div class="col-lg-2 col-xl-2">
            </div>
                <div id='row3checkboxes' class="col-lg-10 col-xl-10">
                </div>
            </div>
        </div>
    </div>
<script>
let pc0;
let figs = new Array();
let legendLabels = new Array(); //the labels in the selection row
const labels = ["selected and of interest", "selected and not of interest",
    "not selected but of interest", "not selected and not of interest" ];

// TODO:: can we make the outcome specific brushes both
// TODO:: globol and dynamic?
let parcoord_brushed = new Set(); // set of brushed items in parcoords
let figs_brushed = new Array();
let allKeys;

const data = $data

const checkboxRow = document.getElementById("row3");
console.log(checkboxRow)
buildSelectionRow();

Set.prototype.intersection = function(setB) {
    var intersection = new Set();
    for (var elem of setB) {
        if (this.has(elem)) {
            intersection.add(elem);
        }
    }
    return intersection;
};

Set.prototype.difference = function(setB) {
    var difference = new Set(this);
    for (var elem of setB) {
        difference.delete(elem);
    }
    return difference;
};

Set.prototype.difference = function(setB) {
    var difference = new Set(this);
    for (var elem of setB) {
        difference.delete(elem);
    }
    return difference;
};

Set.prototype.addEach = function(collection) {
    for (let element of collection){
        this.add(element)
    }
};

processData(data)

function processData(data) {
    // http://stackoverflow.com/questions/16265123/resize-svg-when-window-is-resized-in-d3-js
    // http://blog.webkid.io/responsive-chart-usability-d3/
    console.log(data)
    const uncertainties = Object.keys(data[0].experiment);
    const outcomes = Object.keys(data[0].outcomes);

    const dimensions = {};
    const checkedDimensions = {};
    for (i=0; i<uncertainties.length; i++){
        dimensions[uncertainties[i]] = {title: uncertainties[i], type:"number"};
        checkedDimensions[uncertainties[i]] = {title: uncertainties[i], type:"number"};
    }

    // TODO:: I create the key four times, this is silly
    const experiments = data.map(function(d){
        const experiment = d.experiment;
        experiment.key = "exp"+d.key;
        return experiment});

    allKeys = new Set(data.map(function(d){return "exp"+d.key}));
    parcoord_brushed.addEach(allKeys);

    pc0 = d3.parcoords()("#example")
        .data(experiments)
        .composite("darken")
        .margin({ top: 60, left: 0, bottom: 60, right: 0 })
        .dimensions(checkedDimensions)
        .shadows()
        .mode("queue")
        .rotateLabels(enabled=false)
        .render()
        .brushMode("1D-axes");

    pc0.alphaOnBrushed(1);
    pc0.ticks(2)

    const axes = pc0.svg.selectAll(".axis")[0];
    const textlabels = pc0.svg.selectAll('.resize')
        .append("text")
        .attr("dy", function(n, i){
            if (n=='s'){
                return "+15";
            }
            if (n=='n'){
                return "-5";
            }});

    pc0.on("brush", function(d){
        //lower label TODO:: nog wel naar beneden verplaatsen
        let lower, upper, index, label, brushes, brush;
        brushes = pc0.brushExtents();

        textlabels
            .text(function(n,i){
                // n is upper (s) or lower (n)
                // i is index of axis

                index = Math.floor(i/2);
                label = axes[index].__data__;

                brush = brushes[label];
                if(brush){
                    if (n=='s'){
                        return brush[0].toFixed(2);
                    }
                    if (n=='n'){
                        return brush[1].toFixed(2);
                    }
                }
            })
            .attr("text-anchor", "middle");

        parcoord_brushed.clear();
        parcoord_brushed.addEach(new Set(d.map(function(d){return d.key;})));
        redraw();
    });

    // TODO:; perhaps have an object for each figure instead of index based
    // TODO:: alignment in arrays
    let outcomeData;
    let selectionSet;
    let fig;
    for (let outcome of outcomes){
        outcomeData = data.map(function(d){
            return {key: "exp"+d.key, values:d.outcomes[outcome]}
        });

        selectionSet = new Set();
        selectionSet.addEach(allKeys);

        fig = drawBeeswarm(outcomeData, outcome, selectionSet);
        figs.push(fig);
        figs_brushed.push(selectionSet);
    }

    populateCheckBoxes(uncertainties, dimensions, checkedDimensions);
}

function populateCheckBoxes(uncertainties, dimensions, checkedDimensions){
    const input = document.getElementById("input");
    for (let unc of uncertainties) {
        let div = document.createElement('div');
        let label= document.createElement("label");
        let checkbox = document.createElement('input');

        div.className = "form-check";
        label.className = "form-check-input";

        checkbox.type = "checkbox";
        checkbox.name = unc;
        checkbox.value = true;
        checkbox.checked = true;
        checkbox.onchange = function(){
            this.checked;
            if (this.checked){
                checkedDimensions[this.name] = dimensions[this.name]
            } else {
                delete checkedDimensions[this.name]
            }
            pc0.dimensions(checkedDimensions)
                .render().updateAxes();
        };

        label.appendChild(checkbox);
        label.appendChild(document.createTextNode(unc));

        input.appendChild(label);
        input.appendChild(document.createElement("br"));
    }

    return input
}

function drawBeeswarm(data, outcomeName, selectionSet){
    let textlabels; //for labels of brush
    
    console.log(outcomeName);
    console.log(data); 

    // create a new row
    // the actual modifications of the dom
    const row = d3.select(".container").append("div")
        .attr("class", "row");
    const column1 = row.append("div").attr("class", "col-lg-2 col-xl-2");
    const column2 = row.append("div").attr("class", "col-lg-8 col-xl-8");
    const column3 = row.append("div").attr("class", "col-lg-2 col-xl-2");

    const margin = {top: 20, right: 20, bottom: 40, left: 50};
    const columnWidth = column2[0][0].offsetWidth; // from style sheet
    const rowHeight = 300 + margin.top + margin.bottom; // our choice

    const plotWidth = columnWidth - margin.left - margin.right;
    const plotHeight = rowHeight - margin.top - margin.bottom;

    // scales
    const y = d3.scale.linear()
        .range([plotHeight, 0])
        .domain([d3.min(data, function (d) {
                        return d.values}),
                 d3.max(data, function (d) {
                        return d.values})]);

    const yAxis = d3.svg.axis().scale(y)
        .orient("left").ticks(5);

    const swarm = d3.beeswarm()
        .data(data)
        .distributeOn(function(d){
            return y(d.values);
        })
        .orientation('vertical')
        .side('symetric');
    const beeswarmArrangement =	swarm.arrange();

    // append svg
    const svg = column2.append("svg")
        .attr("width", columnWidth)
        .attr("height", rowHeight)
        .attr("transform",
            "translate(" + margin.left + "," + margin.top + ")");

    const nodeContainer = svg.append("g")
        .attr("id", "node-container");

    const node = nodeContainer.selectAll("circle")
        .data(beeswarmArrangement)
        .enter()
            .append("circle")
                .attr('cx', function(bee) { return bee.x+plotWidth/2; })
                .attr('cy', function(bee) { return bee.y; })
                .attr('r', 4)
                .style('fill', "steelblue");

    // Add the Y Axis
    const svgYaxis = svg.append("g")
        .attr("class", "y axis")
        .attr("transform", "translate(" + plotWidth/3 + ",0)")
        .call(yAxis);

    // add a label to the y-axis
    svg.append("text")
        .attr("transform", "rotate(-90)")
        .attr("y", 0.175*plotWidth)
        .attr("x",0 - (plotHeight / 2))
        .attr("dy", "1em")
        .style("text-anchor", "middle")
        .text(outcomeName);

    // add a brush option to the plot
    const brush = d3.svg.brush()
        .y(y)
        .on("brushstart", function() {
            if(d3.event.sourceEvent !== null) {
                d3.event.sourceEvent.stopPropagation();
            }
        })
        .on("brush", function() {
            const extent = brush.extent();
            selectionSet.clear();

            for (let i=0; i<data.length; i++){
                datum = data[i];
                if (datum.values >= extent[0] && datum.values <= extent[1]) {
                    selectionSet.add(datum.key);
                }
            }

            textlabels
                .text(function(n){
                    // n is upper (s) or lower (n)
                    if (n=='s'){
                        return extent[0].toFixed(2);
                    }
                    if (n=='n'){
                        return extent[1].toFixed(2);
                    }
                })
                .attr("text-anchor", "middle");
            redraw();
        })
        .on("brushend", function() {
            const extent = brush.extent();
            if(extent[0] == extent[1]){
                //checkboxRow.hide();
                selectionSet.clear();
                selectionSet.addEach(allKeys);
            }
            //else {
                //checkboxRow.show();
            //}

            redraw();
        });

    // Add and store a brush for each axis.
    const svgBrush = svgYaxis.append("g")
        .attr("class", "brush")
        .call(brush)
        .selectAll("rect")
        .style("visibility", null)
        .attr("x", -15)
        .attr("width", 30);

    textlabels = svgYaxis.selectAll('.resize')
        .append("text")
        .attr("dy", function(n, i){
            if (n=='s'){
                return "+15";
            }
            if (n=='n'){
                return "-5";
            }});


    return svg;
}

function redraw(){
    // make the four possible classes

    let ofInterest = new Set();
    ofInterest.addEach(allKeys);
    for (let selection of figs_brushed){
        ofInterest = ofInterest.intersection(selection);
    }

    const notOfInterest = allKeys.difference(ofInterest);
    const selected = parcoord_brushed;
    const notSelected = allKeys.difference(selected);

    const soi = selected.intersection(ofInterest);
    const nsoi = notSelected.intersection(ofInterest);
    const snoi = selected.intersection(notOfInterest);
    const nsnoi = notSelected.intersection(notOfInterest);

    for (let fig of figs){
        fig.selectAll("circle")
            .style("fill", function(d){
                let key = d.datum.key;
                if (soi.has(key)){
                    return "#4daf4a";
                } else if (nsoi.has(key)){
                    return "#377eb8";
                } else if (snoi.has(key)) {
                    return "#e41a1c";
                } else {
                    return "grey";
                }})
            .style("visibility", function(d){
                // console.log(document.getElementById("soi"));
                let key = d.datum.key;
                if (soi.has(key) && document.getElementById("soi").checked){
                    return "visible";
                } else if (nsoi.has(key) && document.getElementById("nsoi").checked){
                    return "visible";
                } else if (snoi.has(key) && document.getElementById("snoi").checked) {
                    return "visible";
                } else if (nsnoi.has(key) && document.getElementById("nsnoi").checked){
                    return "visible";
                } else{
                    return "hidden";
                }});
    }


    // hidden can be done using alpha
    pc0
        .color(function(d){
            let key = d.key;
            if (soi.has(key) && document.getElementById("soi").checked){
                return "#4daf4a";
            } else if (nsoi.has(key) && document.getElementById("nsoi").checked){
                return "#377eb8";
            } else if (snoi.has(key) && document.getElementById("snoi").checked) {
                return "#e41a1c";
            } else if (nsnoi.has(key) && document.getElementById("nsnoi").checked){
                return "grey";
            } else{
                return "#00000000";
            }})
        .render();

    let legendLabel, label, set;
    let sets = [soi, snoi, nsoi, nsnoi]
    for (let i=0; i<legendLabels.length; i++){
        legendLabel = legendLabels[i];
        label = labels[i]
        set = sets[i];
        legendLabel.nodeValue = label + " (" + set.size + ")"
    }

}

function buildSelectionRow(){
    const column = document.getElementById("row3checkboxes");
    const ids = ["soi", "snoi", "nsoi", "nsnoi"];
    const colors = ["#4daf4a",
                    "#e41a1c",
                    "#377eb8",
                    "grey"];

    let i, text, item, div, label, checkbox, id, color, textLabel;
    for (i=0; i<labels.length; i++){
        text = labels[i];
        id = ids[i];
        color = colors[i]

        div = document.createElement("div");
        div.className = "form-check form-check-inline";

        label = document.createElement("label");
        label.className = "form-check-label";

        checkbox = document.createElement('input');
        checkbox.className = "form-check-input";
        checkbox.type = "checkbox";
        checkbox.checked = true;
        checkbox.name = text;
        checkbox.id = id;
        checkbox.onchange = function(){
            redraw();
        };

        label.appendChild(checkbox);

        //TODO: use custom styling on checkbox instead
        //TODO: or move this functionality to svg legend as part of parcoords fig
        textLabel = document.createTextNode(text);
        label.style.color = color;
        label.appendChild(textLabel);
        legendLabels.push(textLabel);

        div.appendChild(label);
        column.appendChild(div);
    }

    //checkboxRow.hide();
}

</script>
</body>
</html>
''')


def data_to_string(experiments, outcomes):
    '''format a pandas data frame in a string of dicts, with keys in the order
    they appear in the dataframe
    
    Parameters
    ----------
    data  : pd.Dataframe instance
    
    Returns
    -------
    str
    
    '''
    experiment_dicts = experiments.to_dict(orient='records')

    outcome_dicts = []
    for i in range(experiments.shape[0]):
        entry = {key:value[i] for key, value in outcomes.items()}
        outcome_dicts.append(entry)

    data = []
    for i, entry in enumerate(zip(experiment_dicts, outcome_dicts)):
        experiment, outcome = entry
            
        # might be done more elegantly at the dataframe level
#         transformed_experiment = {}
#         for key, value in experiment.items():
#             if not isinstance(value, numbers.Number):
#                 value = str(value)
#             elif isinstance(value, bool):
#                 value = str(value)
#             transformed_experiment[key] = value
#         experiment = transformed_experiment
        
        entry = {'key':str(i), 'experiment':experiment, 'outcomes':outcome}
        data.append(entry)
        
    return json.dumps(data)

class generate_html(object):
    '''
    
    Parameters
    ----------
    x : pandas dataframe
    y : dict of numpy arrays
    fn : str, optional
         file name, if not provided a temporary file is used
    
    '''
    
    # we need to keep a record of all the tempfiles, otherwise the
    # tempfiles are removed after the call, resulting in a 404 error
    tempfiles = []
    
    def __call__(self, x, y, fn=None):
        if fn is None:
            fh = tempfile.NamedTemporaryFile(suffix='.html', 
                        dir='.',
                        mode='w+t')
        else:
            fh = open(fn, 'w')
        
        data = data_to_string(x, y)

        formatted = content.substitute({'data':data})
        fh.write(formatted)
        fh.seek(0)
    
        if fn is None:
            self.tempfiles.append(fh)
        
        file_loc = './{}'
        file_path = file_loc.format(os.path.basename(fh.name))
        
        return IFrame(file_path, 1000,800)

# instantiate the class, and make it look as if it is a simple function
generate_html = generate_html()


In [10]:
from ema_workbench import load_results

In [11]:
results = load_results('./data/100 runs wcm.tar.gz')


In [13]:
experiments, outcomes = results
x = pd.DataFrame.from_records(experiments)
y = outcomes

generate_html(x, y)


In [ ]:
experiments = pd.DataFrame.from_records(experiments)
experiment_dicts = experiments.to_dict(orient='records')

outcome_dicts = []
for i in range(experiments.shape[0]):
    entry = {key:value[i] for key, value in outcomes.items()}
    outcome_dicts.append(entry)

data = []
for i, entry in enumerate(zip(experiment_dicts, outcome_dicts)):
    experiment, outcome = entry
    entry = {'key':str(i), 'experiment':experiment, 'outcomes':outcome}
    data.append(entry)

In [5]:
import json

In [8]:
json.dumps(True)

'true'